In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


''

In [2]:
import pymongo

from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')

In [3]:
db = client['test-database']
collection = db.test_collection

In [4]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test-database'), 'test_collection')